In [6]:
import pandas as pd
import uuid
from sqlalchemy import create_engine
from sqlalchemy import text
from config import DATABASE_URL 

In [4]:
file = pd.read_csv('Sample Superstore.csv')

In [6]:
file

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,28/05/2012,30/05/2012,1.320000,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,07/07/2010,08/07/2010,4.560000,4,13.01,88522
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,27/07/2011,28/07/2011,-47.640000,7,49.92,88523
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,27/07/2011,28/07/2011,-30.510000,7,41.64,88523
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,...,West,Washington,Anacortes,98221,27/07/2011,27/07/2011,998.202300,8,1446.67,88523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9421,20275,Critical,0.06,35.89,14.72,3402,Frederick Cole,Regular Air,Consumer,Office Supplies,...,East,West Virginia,Charleston,25314,14/05/2013,15/05/2013,137.860000,13,447.87,87532
9422,20276,Critical,0.00,3.34,7.49,3402,Frederick Cole,Regular Air,Consumer,Office Supplies,...,East,West Virginia,Charleston,25314,14/05/2013,14/05/2013,-39.070000,3,13.23,87532
9423,24491,Not Specified,0.08,550.98,45.70,3402,Frederick Cole,Delivery Truck,Consumer,Furniture,...,East,West Virginia,Charleston,25314,12/09/2013,14/09/2013,-1225.029097,4,2215.93,87533
9424,25914,High,0.10,105.98,13.99,3403,Tammy Buckley,Express Air,Consumer,Furniture,...,West,Wyoming,Cheyenne,82001,08/02/2010,11/02/2010,349.485000,5,506.50,87530


In [8]:
# Create a dictionary to store UUIDs for unique customer combinations
unique_customers = {}

# Function to get or create UUID
def get_customer_guid(row):
    # Create a tuple of the key items
    key = (row['Customer ID'])
    if key not in unique_customers:
        # Assign new UUID if key is not found
        unique_customers[key] = str(uuid.uuid4())
    return unique_customers[key]

# Apply the function to each row in the DataFrame
file['CustomerGUID'] = file.apply(get_customer_guid, axis=1)

In [10]:
# Create a dictionary to store product IDs for unique combinations
unique_products = {}
current_product_id = 1  # Start product ID numbering from 1

# Function to get or create a product ID
def get_product_id(row):
    global current_product_id
    # Create a tuple of the key items
    key = (row['Product Category'], row['Product Sub-Category'], row['Product Container'])
    if key not in unique_products:
        # Assign new product ID if key is not found and increment the ID
        unique_products[key] = current_product_id
        current_product_id += 1
    return unique_products[key]

# Apply the function to each row in the DataFrame
file['Product ID'] = file.apply(get_product_id, axis=1)

In [12]:
# Create a dictionary to store UUIDs for unique customer combinations
unique_products_uguid = {}

# Function to get or create UUID
def get_product_guid(row):
    # Create a tuple of the key items
    key = (row['Product ID'])
    if key not in unique_products_uguid:
        # Assign new UUID if key is not found
        unique_products_uguid[key] = str(uuid.uuid4())
    return unique_products_uguid[key]

# Apply the function to each row in the DataFrame
file['ProductGUID'] = file.apply(get_product_guid, axis=1)

In [14]:
# Create a dictionary to store product IDs for unique combinations
unique_orders = {}
current_order_id = 1  # Start product ID numbering from 1

# Function to get or create a product ID
def get_order_id(row):
    global current_order_id
    # Create a tuple of the key items
    key = (row['CustomerGUID'], row['Order Date'], row['Order Priority'])
    if key not in unique_orders:
        # Assign new product ID if key is not found and increment the ID
        unique_orders[key] = current_order_id
        current_order_id += 1
    return unique_orders[key]

# Apply the function to each row in the DataFrame
file['Order ID'] = file.apply(get_order_id, axis=1)

In [16]:
# Create a dictionary to store UUIDs for unique customer combinations
unique_orders_uguid = {}

# Function to get or create UUID
def get_order_guid(row):
    # Create a tuple of the key items
    key = (row['Order ID'])
    if key not in unique_orders_uguid:
        # Assign new UUID if key is not found
        unique_orders_uguid[key] = str(uuid.uuid4())
    return unique_orders_uguid[key]

# Apply the function to each row in the DataFrame
file['OrderGUID'] = file.apply(get_order_guid, axis=1)

In [18]:
# Create a dictionary to store product IDs for unique combinations
unique_address = {}
current_address_id = 1  # Start product ID numbering from 1

# Function to get or create a product ID
def get_address_id(row):
    global current_address_id
    # Create a tuple of the key items
    key = (row['Region'], row['State or Province'], row['City'], row['Postal Code'], row['CustomerGUID'])
    if key not in unique_address:
        # Assign new product ID if key is not found and increment the ID
        unique_address[key] = current_address_id
        current_address_id += 1
    return unique_address[key]

# Apply the function to each row in the DataFrame
file['Address ID'] = file.apply(get_address_id, axis=1)

In [20]:
# Create a dictionary to store UUIDs for unique customer combinations
unique_address_uguid = {}

# Function to get or create UUID
def get_address_guid(row):
    # Create a tuple of the key items
    key = (row['Address ID'])
    if key not in unique_address_uguid:
        # Assign new UUID if key is not found
        unique_address_uguid[key] = str(uuid.uuid4())
    return unique_address_uguid[key]

# Apply the function to each row in the DataFrame
file['AddressGUID'] = file.apply(get_address_guid, axis=1)

In [22]:
# Create a dictionary to store product IDs for unique combinations
unique_shipment = {}
current_shipment_id = 1  # Start product ID numbering from 1

# Function to get or create a product ID
def get_shipment_id(row):
    global current_shipment_id
    # Create a tuple of the key items
    key = (row['Ship Mode'], row['Ship Date'], row['AddressGUID'])
    if key not in unique_shipment:
        # Assign new product ID if key is not found and increment the ID
        unique_shipment[key] = current_shipment_id
        current_shipment_id += 1
    return unique_shipment[key]

# Apply the function to each row in the DataFrame
file['Shipping ID'] = file.apply(get_shipment_id, axis=1)

In [24]:
# Create a dictionary to store UUIDs for unique customer combinations
unique_shipment_uguid = {}

# Function to get or create UUID
def get_shipment_guid(row):
    # Create a tuple of the key items
    key = (row['Shipping ID'])
    if key not in unique_shipment_uguid:
        # Assign new UUID if key is not found
        unique_shipment_uguid[key] = str(uuid.uuid4())
    return unique_shipment_uguid[key]

# Apply the function to each row in the DataFrame
file['ShippingGUID'] = file.apply(get_shipment_guid, axis=1)

In [26]:
# Create a dictionary to store product IDs for unique combinations
unique_sales = {}
current_sales_id = 1  # Start product ID numbering from 1

# Function to get or create a product ID
def get_address_id(row):
    global current_sales_id
    # Create a tuple of the key items
    key = (row['OrderGUID'])
    if key not in unique_sales:
        # Assign new product ID if key is not found and increment the ID
        unique_sales[key] = current_sales_id
        current_sales_id += 1
    return unique_sales[key]

# Apply the function to each row in the DataFrame
file['Sales ID'] = file.apply(get_address_id, axis=1)

In [28]:
# Create a dictionary to store UUIDs for unique customer combinations
unique_sales_uguid = {}

# Function to get or create UUID
def get_sales_guid(row):
    # Create a tuple of the key items
    key = (row['Sales ID'])
    if key not in unique_sales_uguid:
        # Assign new UUID if key is not found
        unique_sales_uguid[key] = str(uuid.uuid4())
    return unique_sales_uguid[key]

# Apply the function to each row in the DataFrame
file['SalesGUID'] = file.apply(get_sales_guid, axis=1)

In [30]:
file

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,CustomerGUID,Product ID,ProductGUID,OrderGUID,Address ID,AddressGUID,Shipping ID,ShippingGUID,Sales ID,SalesGUID
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,7904e29e-6919-454f-b9ba-fe674ce5b687,1,a828de89-cc82-45d4-bf6e-a5b176032f59,6b82c529-a2eb-4efc-81ba-6ff211f6fa13,1,7589ebad-3d13-4590-8c41-3b539607a503,1,a70b7dce-8c66-4c6b-ae3f-dbf9461afb56,1,2931800d-193c-4ca8-9801-475f896303bc
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,8b604a0b-6dd9-4917-b8eb-17796b5df01c,2,4ae74995-1034-481b-813d-ead1c7292f94,5990b4b3-01f7-42e2-bfc0-9e82c7fddbee,2,5b380529-aef3-48db-99f3-f75dc07ff3d3,2,469561a8-4a8d-428a-b669-0760b4a5a459,2,91b2f311-5873-41a3-8e75-559695a6568d
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,8b604a0b-6dd9-4917-b8eb-17796b5df01c,3,eebe042a-6a13-4133-b768-03d7e64c9369,154cb7c0-b515-4b1f-86e0-aa29d1c06d16,2,5b380529-aef3-48db-99f3-f75dc07ff3d3,3,9713e655-ff33-4e64-aec5-0a6f233cc5d9,3,eb8ca898-d5ae-4d65-8d46-7ba349ded4fb
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,...,8b604a0b-6dd9-4917-b8eb-17796b5df01c,4,f4c0adb5-fc72-4478-a4f6-d235fa5daa65,154cb7c0-b515-4b1f-86e0-aa29d1c06d16,2,5b380529-aef3-48db-99f3-f75dc07ff3d3,4,fc3ad86f-45c2-4169-8685-99084b1ac812,3,eb8ca898-d5ae-4d65-8d46-7ba349ded4fb
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,...,8b604a0b-6dd9-4917-b8eb-17796b5df01c,5,8fc5df08-168f-4cbf-9b68-2580389ae0d7,154cb7c0-b515-4b1f-86e0-aa29d1c06d16,2,5b380529-aef3-48db-99f3-f75dc07ff3d3,5,82631fd6-887c-4956-970f-12ca9b4fd21b,3,eb8ca898-d5ae-4d65-8d46-7ba349ded4fb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9421,20275,Critical,0.06,35.89,14.72,3402,Frederick Cole,Regular Air,Consumer,Office Supplies,...,7b1c440a-5f5a-4a89-8ba4-4142c721dcdf,12,bc1c3f2e-218d-4ac0-a5dd-175cb5fc121f,9fe79a74-e643-4715-8217-c4c44af6902d,2702,e8cd7853-6422-4e6d-b234-c28a7e309ea1,8886,7cca8b02-7a35-45c6-b610-a58f48629143,7163,c4071710-13b3-415e-a8f1-e2de6ecc4785
9422,20276,Critical,0.00,3.34,7.49,3402,Frederick Cole,Regular Air,Consumer,Office Supplies,...,7b1c440a-5f5a-4a89-8ba4-4142c721dcdf,2,4ae74995-1034-481b-813d-ead1c7292f94,9fe79a74-e643-4715-8217-c4c44af6902d,2702,e8cd7853-6422-4e6d-b234-c28a7e309ea1,8887,eca48541-fc88-44cc-92e4-87e72e9c45c1,7163,c4071710-13b3-415e-a8f1-e2de6ecc4785
9423,24491,Not Specified,0.08,550.98,45.70,3402,Frederick Cole,Delivery Truck,Consumer,Furniture,...,7b1c440a-5f5a-4a89-8ba4-4142c721dcdf,11,f8ac426e-bd1b-4289-af7b-ee392b687a2f,d91bd2e1-6c58-4128-a2a4-81986a02da74,2702,e8cd7853-6422-4e6d-b234-c28a7e309ea1,8888,7db87d16-9f98-450a-8e08-1c0cf4d6adeb,7164,1f0d77b6-fab1-40b8-95dc-168ed4c1c6d5
9424,25914,High,0.10,105.98,13.99,3403,Tammy Buckley,Express Air,Consumer,Furniture,...,b0011950-c4b0-4c3b-a1a9-b56d1ecbb573,35,a089bdea-dcf9-49c4-8ce4-fd98c3c9b3ef,299511a4-cf09-4c84-ae8e-8cb8eedcad78,2703,15ddbcdb-79a1-4f2b-be6c-c0d7e8ef1aea,8889,8f0e2888-5d2e-4159-a932-277738428afa,7165,b6636219-df38-4329-b4f0-0bd50360f0c6


In [32]:
customer_df = file[['CustomerGUID', 'Customer ID', 'Customer Name', 'Customer Segment']].drop_duplicates(subset=['Customer ID'])

In [34]:
product_df = file [['ProductGUID', 'Product ID', 'Product Category', 'Product Sub-Category', 'Product Container']].drop_duplicates(subset=['Product ID'])

In [36]:
shipping_df = file[['ShippingGUID', 'Shipping ID', 'Ship Date', 'Shipping Cost', 'Ship Mode', 'AddressGUID']].drop_duplicates(subset=['Shipping ID'])

In [38]:
order_df = file[['OrderGUID', 'Order ID', 'Order Date', 'Order Priority', 'CustomerGUID', 'ShippingGUID', 'ProductGUID']].drop_duplicates(subset=['Order ID'])

In [40]:
address_df = file [['AddressGUID', 'Address ID', 'Region', 'State or Province', 'City', 'Postal Code', 'CustomerGUID']].drop_duplicates(subset=['Address ID'])

In [42]:
sales_df = file[['SalesGUID', 'Sales ID', 'Profit', 'Sales', 'Quantity ordered new', 'Product Base Margin', 'Discount', 'Unit Price', 'OrderGUID', 'ProductGUID']].drop_duplicates(subset=['Sales ID'])

In [44]:
customer_df = customer_df.rename(columns = {
    'CustomerGUID': 'CustomerGUID',
    'Customer ID': 'CustomerID',
    'Customer Name': 'FullName',
    'Customer Segment': 'Segment'
})

In [46]:
product_df = product_df.rename(columns = {
    'ProductGUID': 'ProductGUID',
    'Product ID': 'ProductID',
    'Product Category': 'Category',
    'Product Sub-Category': 'SubCategory',
    'Product Container': 'Container'
})

In [48]:
shipping_df = shipping_df.rename(columns = {
    'ShippingGUID': 'ShippingGUID',
    'Shipping ID': 'ShippingID',
    'Ship Date': 'DateShipped',
    'Shipping Cost': 'ShippingCost',
    'Ship Mode': 'ShipMode',
    'AddressGUID': 'AddressGUID'
})

In [50]:
order_df = order_df.rename(columns = {
    'OrderGUID': 'OrderGUID',
    'Order ID': 'OrderID',
    'Order Date': 'DateOrdered',
    'Order Priority': 'Priority',
    'CustomerGUID': 'CustomerGUID',
    'ShippingGUID': 'ShippingGUID',
    'ProductGUID': 'ProductGUID'
})

In [52]:
address_df = address_df.rename(columns = {
    'AddressGUID': 'AddressGUID',
    'Address ID': 'AddressID',
    'Region': 'Region',
    'State or Province': 'State_Province',
    'City': 'City',
    'Postal Code': 'PostalCode',
    'CustomerGUID': 'CustomerGUID'
})

In [54]:
sales_df = sales_df.rename(columns = {
    'SalesGUID': 'SalesGUID',
    'Sales ID': 'SalesID',
    'Profit': 'Profit',
    'Sales': 'Sales',
    'Quantity ordered new': 'QuantityOrdered', 
    'Product Base Margin': 'ProductBaseMargin',
    'Discount': 'Discount',
    'Unit Price': 'UnitPrice',
    'OrderGUID': 'OrderGUID',
    'ProductGUID': 'ProductGUID'
})

In [336]:
# Convert the 'DateShipped' to the correct format with dayfirst=True to handle dd/mm/yyyy format
shipping_df['DateShipped'] = pd.to_datetime(shipping_df['DateShipped'], dayfirst=True)

# Format 'DateShipped' to yyyy-mm-dd format for PostgreSQL compatibility
shipping_df['DateShipped'] = shipping_df['DateShipped'].dt.strftime('%Y-%m-%d')

# Save the modified CSV file
shipping_df.to_csv('Shipping.csv', index=False)

/var/folders/_b/ygjd3hwd5wj6hwcczmfy35gh0000gn/T/ipykernel_1837/3867188745.py:2: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  shipping_df['DateShipped'] = pd.to_datetime(shipping_df['DateShipped'], dayfirst=True)


In [56]:
# Convert the 'DateShipped' to the correct format with dayfirst=True to handle dd/mm/yyyy format
order_df['DateOrdered'] = pd.to_datetime(order_df['DateOrdered'], dayfirst=True)

# Format 'DateShipped' to yyyy-mm-dd format for PostgreSQL compatibility
order_df['DateOrdered'] = order_df['DateOrdered'].dt.strftime('%Y-%m-%d')

# Save the modified CSV file
order_df.to_csv('Order.csv', index=False)

In [58]:
customer_df.to_csv('Customer.csv', index = False)
product_df.to_csv('Product.csv', index = False)
shipping_df.to_csv('Shipping.csv', index = False)
order_df.to_csv('Order.csv', index = False)
address_df.to_csv('Address.csv', index = False)
sales_df.to_csv('Sales.csv', index = False)

In [8]:
# Create the connection engine
engine = create_engine(DATABASE_URL, echo=True)

In [194]:
# Correct usage of connection and transaction scopes
def create_customer_table():
    # Open a connection
    with engine.connect() as connection:
        # Start a transaction
        with connection.begin():
            # Prepare the create table command using triple quotes for multi-line text
            create_command = text("""
            CREATE TABLE Customer (
                CustomerGUID UUID PRIMARY KEY,
                CustomerID INT,
                FullName VARCHAR(255),
                Segment VARCHAR(255)
            );
            """)
            # Execute the create command
            connection.execute(create_command)
        # The transaction is committed here automatically when the block ends
    # The connection is closed here automatically when the block ends

# Call the function to execute the command
create_customer_table()

2025-01-04 07:29:29,608 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-04 07:29:29,612 INFO sqlalchemy.engine.Engine 
            CREATE TABLE Customer (
                CustomerGUID UUID PRIMARY KEY,
                CustomerID INT,
                FullName VARCHAR(255),
                Segment VARCHAR(255)
            );
            
2025-01-04 07:29:29,613 INFO sqlalchemy.engine.Engine [generated in 0.00157s] {}
2025-01-04 07:29:29,619 INFO sqlalchemy.engine.Engine COMMIT


In [240]:
# Correct usage of connection and transaction scopes
def create_product_table():
    # Open a connection
    with engine.connect() as connection:
        # Start a transaction
        with connection.begin():
            # Prepare the create table command using triple quotes for multi-line text
            create_command = text("""
            CREATE TABLE Product (
                ProductGUID UUID PRIMARY KEY,
                Category VARCHAR(255),
                SubCategory VARCHAR(255),
                Container VARCHAR(255),
            	ProductID INT
            );
            """)
            # Execute the create command
            connection.execute(create_command)
        # The transaction is committed here automatically when the block ends
    # The connection is closed here automatically when the block ends

# Call the function to execute the command
create_product_table()

2025-01-04 07:52:36,772 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-04 07:52:36,778 INFO sqlalchemy.engine.Engine 
            CREATE TABLE Product (
                ProductGUID UUID PRIMARY KEY,
                Category VARCHAR(255),
                SubCategory VARCHAR(255),
                Container VARCHAR(255),
            	ProductID INT
            );
            
2025-01-04 07:52:36,778 INFO sqlalchemy.engine.Engine [generated in 0.00159s] {}
2025-01-04 07:52:36,791 INFO sqlalchemy.engine.Engine COMMIT


In [84]:
# Correct usage of connection and transaction scopes
def create_address_table():
    # Open a connection
    with engine.connect() as connection:
        # Start a transaction
        with connection.begin():
            # Prepare the create table command using triple quotes for multi-line text
            create_command = text("""
            CREATE TABLE Address (
                AddressGUID UUID PRIMARY KEY,
            	AddressID INT,
                Region VARCHAR(255),
                State_Province VARCHAR(255),
                City VARCHAR(255),
                PostalCode VARCHAR(255),
                CustomerGUID UUID REFERENCES Customer(CustomerGUID)
            );
            """)
            # Execute the create command
            connection.execute(create_command)
        # The transaction is committed here automatically when the block ends
    # The connection is closed here automatically when the block ends

# Call the function to execute the command
create_address_table()

2025-01-04 07:14:20,903 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-04 07:14:20,906 INFO sqlalchemy.engine.Engine 
            CREATE TABLE Address (
                AddressGUID UUID PRIMARY KEY,
            	AddressID INT,
                Region VARCHAR(255),
                State_Province VARCHAR(255),
                City VARCHAR(255),
                PostalCode VARCHAR(255),
                CustomerGUID UUID REFERENCES Customer(CustomerGUID)
            );
            
2025-01-04 07:14:20,908 INFO sqlalchemy.engine.Engine [generated in 0.00155s] {}
2025-01-04 07:14:20,917 INFO sqlalchemy.engine.Engine COMMIT


In [86]:
# Correct usage of connection and transaction scopes
def create_shipping_table():
    # Open a connection
    with engine.connect() as connection:
        # Start a transaction
        with connection.begin():
            # Prepare the create table command using triple quotes for multi-line text
            create_command = text("""
            CREATE TABLE Shipping (
                ShippingGUID UUID PRIMARY KEY,
                ShippingID INT,
                DateShipped TIMESTAMP,
                ShippingCost MONEY,
                ShipMode VARCHAR(255),
                AddressGUID UUID REFERENCES Address(AddressGUID)
            );
            """)
            # Execute the create command
            connection.execute(create_command)
        # The transaction is committed here automatically when the block ends
    # The connection is closed here automatically when the block ends

# Call the function to execute the command
create_shipping_table()

2025-01-04 07:14:52,218 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-04 07:14:52,220 INFO sqlalchemy.engine.Engine 
            CREATE TABLE Shipping (
                ShippingGUID UUID PRIMARY KEY,
                ShippingID INT,
                DateShipped TIMESTAMP,
                ShippingCost MONEY,
                ShipMode VARCHAR(255),
                AddressGUID UUID REFERENCES Address(AddressGUID)
            );
            
2025-01-04 07:14:52,221 INFO sqlalchemy.engine.Engine [generated in 0.00060s] {}
2025-01-04 07:14:52,226 INFO sqlalchemy.engine.Engine COMMIT


In [196]:
# Correct usage of connection and transaction scopes
def create_order_table():
    # Open a connection
    with engine.connect() as connection:
        # Start a transaction
        with connection.begin():
            # Prepare the create table command using triple quotes for multi-line text
            create_command = text("""
            CREATE TABLE "Order" (
                OrderGUID UUID PRIMARY KEY,
                OrderID INT,
                DateOrdered TIMESTAMP,
                Priority VARCHAR(255),
                CustomerGUID UUID REFERENCES Customer(CustomerGUID),
                ShippingGUID UUID REFERENCES Shipping(ShippingGUID),
                ProductGUID UUID REFERENCES Product(ProductGUID)
            );
            """)
            # Execute the create command
            connection.execute(create_command)
        # The transaction is committed here automatically when the block ends
    # The connection is closed here automatically when the block ends

# Call the function to execute the command
create_order_table()

2025-01-04 07:30:09,483 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-04 07:30:09,488 INFO sqlalchemy.engine.Engine 
            CREATE TABLE "Order" (
                OrderGUID UUID PRIMARY KEY,
                OrderID INT,
                DateOrdered TIMESTAMP,
                Priority VARCHAR(255),
                CustomerGUID UUID REFERENCES Customer(CustomerGUID),
                ShippingGUID UUID REFERENCES Shipping(ShippingGUID),
                ProductGUID UUID REFERENCES Product(ProductGUID)
            );
            
2025-01-04 07:30:09,490 INFO sqlalchemy.engine.Engine [generated in 0.00143s] {}
2025-01-04 07:30:09,496 INFO sqlalchemy.engine.Engine COMMIT


In [90]:
# Correct usage of connection and transaction scopes
def create_sales_table():
    # Open a connection
    with engine.connect() as connection:
        # Start a transaction
        with connection.begin():
            # Prepare the create table command using triple quotes for multi-line text
            create_command = text("""
            CREATE TABLE Sales (
                SalesGUID UUID PRIMARY KEY,
                SalesID INT,
                Profit MONEY,
                Sales MONEY,
                QuantityOrdered INT,
                ProductBaseMargin MONEY,
                Discount DECIMAL,
                UnitPrice MONEY,
                OrderGUID UUID REFERENCES "Order"(OrderGUID),
                ProductGUID UUID REFERENCES Product(ProductGUID)
            );
            """)
            # Execute the create command
            connection.execute(create_command)
        # The transaction is committed here automatically when the block ends
    # The connection is closed here automatically when the block ends

# Call the function to execute the command
create_sales_table()

2025-01-04 07:15:44,365 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-04 07:15:44,369 INFO sqlalchemy.engine.Engine 
            CREATE TABLE Sales (
                SalesGUID UUID PRIMARY KEY,
                SalesID INT,
                Profit MONEY,
                Sales MONEY,
                QuantityOrdered INT,
                ProductBaseMargin MONEY,
                Discount DECIMAL,
                UnitPrice MONEY,
                OrderGUID UUID REFERENCES "Order"(OrderGUID),
                ProductGUID UUID REFERENCES Product(ProductGUID)
            );
            
2025-01-04 07:15:44,371 INFO sqlalchemy.engine.Engine [generated in 0.00205s] {}
2025-01-04 07:15:44,378 INFO sqlalchemy.engine.Engine COMMIT


In [204]:
customer_df.columns = [col.lower() for col in customer_df.columns]  # Adjust based on actual column names required

In [206]:
customer_df.to_sql('customer', con=engine, index=False, if_exists='append', schema='public')  # Ensure correct case and schema

2025-01-04 07:34:36,469 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-04 07:34:36,475 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_namespace.nspname = %(nspname_1)s
2025-01-04 07:34:36,476 INFO sqlalchemy.engine.Engine [cached since 202.6s ago] {'table_name': 'customer', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'public'}
2025-01-04 07:34:36,499 INFO sqlalchemy.engine.Engine INSERT INTO public.customer (customerguid, customerid, fullname, segment) VALUES (%(customerguid__0)s, %(customerid__0)s, %(fullname__0)s, %(segment__0)s), (%(customerguid__1)s, %(customerid__1)s, %(fullname__1)s, %(segment__1)s), (%(custom

703

In [258]:
product_df.columns = [col.lower() for col in product_df.columns]  # Adjust based on actual column names required

In [262]:
# Insert the product_df DataFrame into the 'Product' table
product_df.to_sql('product', con=engine, index=False, if_exists='append')

2025-01-04 07:54:22,202 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-04 07:54:22,207 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-01-04 07:54:22,208 INFO sqlalchemy.engine.Engine [cached since 2052s ago] {'table_name': 'product', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-01-04 07:54:22,212 INFO sqlalchemy.engine.Engine INSERT INTO product (productguid, productid, category, subcategory, container) VALUES (%(productguid__0)s, %(productid__0)s, %(category__0)s, %(subcategory__0)s, %(container__0)

46

In [338]:
shipping_df.columns = [col.lower() for col in shipping_df.columns]  # Adjust based on actual column names required

In [348]:
# Insert the shipping_df DataFrame into the 'Shipping' table
shipping_df.to_sql('shipping', con=engine, index=False, if_exists='append')

2025-01-04 08:02:38,063 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-04 08:02:38,071 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-01-04 08:02:38,073 INFO sqlalchemy.engine.Engine [cached since 2548s ago] {'table_name': 'shipping', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-01-04 08:02:38,103 INFO sqlalchemy.engine.Engine INSERT INTO shipping (shippingguid, shippingid, dateshipped, shippingcost, shipmode, addressguid) VALUES (%(shippingguid__0)s, %(shippingid__0)s, %(dateshipped__0)s, %(shipping

890

In [350]:
order_df.columns = [col.lower() for col in order_df.columns]  # Adjust based on actual column names required

In [352]:
# Insert the order_df DataFrame into the 'Order' table
order_df.to_sql('Order', con=engine, index=False, if_exists='append')

2025-01-04 08:02:43,042 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-04 08:02:43,047 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-01-04 08:02:43,048 INFO sqlalchemy.engine.Engine [cached since 2553s ago] {'table_name': 'Order', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-01-04 08:02:43,083 INFO sqlalchemy.engine.Engine INSERT INTO "Order" (orderguid, orderid, dateordered, priority, customerguid, shippingguid, productguid) VALUES (%(orderguid__0)s, %(orderid__0)s, %(dateordered__0)s, %(priority__

166

In [344]:
address_df.columns = [col.lower() for col in address_df.columns]  # Adjust based on actual column names required

In [346]:
# Insert the address_df DataFrame into the 'Address' table
address_df.to_sql('address', con=engine, index=False, if_exists='append')

2025-01-04 08:02:31,457 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-04 08:02:31,460 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-01-04 08:02:31,461 INFO sqlalchemy.engine.Engine [cached since 2541s ago] {'table_name': 'address', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-01-04 08:02:31,474 INFO sqlalchemy.engine.Engine INSERT INTO address (addressguid, addressid, region, state_province, city, postalcode, customerguid) VALUES (%(addressguid__0)s, %(addressid__0)s, %(region__0)s, %(state_provinc

703

In [14]:
def add_state_code_column():
    # Open a connection
    with engine.connect() as connection:
        # Start a transaction
        with connection.begin():
            # Prepare the ALTER TABLE command to add a new column
            alter_command = text("""
            ALTER TABLE Address
            ADD COLUMN State_Code VARCHAR(10);
            """)
            # Execute the ALTER command
            connection.execute(alter_command)
        # The transaction is committed here automatically when the block ends
    # The connection is closed here automatically when the block ends

# Call the function to execute the command
add_state_code_column()

2025-01-12 11:06:02,218 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-12 11:06:02,229 INFO sqlalchemy.engine.Engine 
            ALTER TABLE Address
            ADD COLUMN State_Code VARCHAR(10);
            
2025-01-12 11:06:02,229 INFO sqlalchemy.engine.Engine [generated in 0.00232s] {}
2025-01-12 11:06:02,416 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
state_code_mapping = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

In [18]:
def update_state_codes(state_code_mapping):
    # Open a connection
    with engine.connect() as connection:
        # Start a transaction
        with connection.begin():
            # Iterate over the state_code dictionary
            for state_name, state_code in state_code_mapping.items():
                # Prepare the UPDATE statement to set the state code
                update_command = text("""
                UPDATE Address
                SET State_Code = :state_code
                WHERE State_Province = :state_name;
                """)
                # Execute the UPDATE command with parameters
                connection.execute(update_command, {'state_code': state_code, 'state_name': state_name})
        # The transaction is committed here automatically when the block ends
    # The connection is closed here automatically when the block ends

# Call the function to update the state codes
update_state_codes(state_code_mapping)

2025-01-12 11:06:20,578 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-12 11:06:20,582 INFO sqlalchemy.engine.Engine 
                UPDATE Address
                SET State_Code = %(state_code)s
                WHERE State_Province = %(state_name)s;
                
2025-01-12 11:06:20,583 INFO sqlalchemy.engine.Engine [generated in 0.00122s] {'state_code': 'AL', 'state_name': 'Alabama'}
2025-01-12 11:06:20,589 INFO sqlalchemy.engine.Engine 
                UPDATE Address
                SET State_Code = %(state_code)s
                WHERE State_Province = %(state_name)s;
                
2025-01-12 11:06:20,590 INFO sqlalchemy.engine.Engine [cached since 0.008373s ago] {'state_code': 'AK', 'state_name': 'Alaska'}
2025-01-12 11:06:20,592 INFO sqlalchemy.engine.Engine 
                UPDATE Address
                SET State_Code = %(state_code)s
                WHERE State_Province = %(state_name)s;
                
2025-01-12 11:06:20,592 INFO sqlalchemy.engine.Engine [cach

In [394]:
sales_df.columns = [col.lower() for col in sales_df.columns]  # Adjust based on actual column names required

In [398]:
# Insert the sales_df DataFrame into the 'Sales' table
sales_df.to_sql('sales', con=engine, index=False, if_exists='append')

2025-01-04 08:07:18,981 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-04 08:07:18,989 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-01-04 08:07:18,990 INFO sqlalchemy.engine.Engine [cached since 2828s ago] {'table_name': 'sales', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-01-04 08:07:19,048 INFO sqlalchemy.engine.Engine INSERT INTO sales (salesguid, salesid, profit, sales, quantityordered, productbasemargin, discount, unitprice, orderguid, productguid) VALUES (%(salesguid__0)s, %(salesid__0)s, %(

166

In [10]:
query = "Select * From Customer;"

df_customer = pd.read_sql_query(query, con=engine)

df_customer

2025-01-11 13:57:19,358 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-01-11 13:57:19,360 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-11 13:57:19,364 INFO sqlalchemy.engine.Engine select current_schema()
2025-01-11 13:57:19,365 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-11 13:57:19,368 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-01-11 13:57:19,369 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-11 13:57:19,371 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-11 13:57:19,372 INFO sqlalchemy.engine.Engine Select * From Customer;
2025-01-11 13:57:19,373 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-11 13:57:19,420 INFO sqlalchemy.engine.Engine ROLLBACK


,customerguid,customerid,fullname,segment
0,66379f73-c216-4450-bdd1-6f19512d5411,2,Janice Fletcher,Corporate
1,148d17dd-43f0-4855-9704-96257bf93292,3,Bonnie Potter,Corporate
2,12ac1694-41bc-48be-b1bc-a243c5f33c3d,5,Ronnie Proctor,Home Office
3,c3778522-0212-4691-8204-40f6b716de79,6,Dwight Hwang,Home Office
4,19dadff2-cfd4-4e8c-b848-3066daea6f85,7,Leon Gill,Home Office
...,...,...,...,...
2698,7d0f1c46-6ace-46b0-aabe-ed12cd87f006,3398,Marc McDaniel,Corporate
2699,3aff70a4-1990-4f9a-abb1-67a9db1e85bb,3399,Marvin Reid,Small Business
2700,5dd3dbf4-a1a1-49c3-aa1a-21c94925adb3,3400,Florence Gold,Small Business
2701,3782402f-9d91-41fe-ab15-63f25972f082,3402,Frederick Cole,Consumer


In [12]:
query = "Select * From Product;"

df_product = pd.read_sql_query(query, con=engine)

df_product

2025-01-11 13:57:20,870 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-11 13:57:20,872 INFO sqlalchemy.engine.Engine Select * From Product;
2025-01-11 13:57:20,873 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-11 13:57:20,876 INFO sqlalchemy.engine.Engine ROLLBACK


,productguid,category,subcategory,container,productid
0,e4ef7fb5-9a0a-4599-b82d-242ec77ebc52,Office Supplies,Labels,Small Box,1
1,ed8aa96c-b68f-42cc-b963-3680b23a92c4,Office Supplies,Pens & Art Supplies,Wrap Bag,2
2,738e6531-e2fb-4fdc-af0b-b58389f73e57,Office Supplies,Paper,Small Box,3
3,a5394cfa-30a4-4615-962a-df36baf93f85,Office Supplies,"Scissors, Rulers and Trimmers",Small Pack,4
4,1f08028e-660f-4bf9-af1b-c31274105e7c,Technology,Telephones and Communication,Small Box,5
5,fef17e15-34a8-4859-88da-53fd3d3b55c5,Technology,Office Machines,Jumbo Drum,6
6,38d5a9ce-872b-42f2-ace5-a04dff5365bc,Furniture,Chairs & Chairmats,Jumbo Drum,7
7,1e06b708-2c1a-4a1e-96eb-de2bddf080b9,Office Supplies,Appliances,Small Box,8
8,253683ef-b61a-4996-9f07-34e8dece887a,Technology,Telephones and Communication,Wrap Bag,9
9,31b7421d-aee4-474f-b7a2-866884e6533e,Furniture,Bookcases,Jumbo Box,10


In [10]:
query = "Select * From Address;"

df_address = pd.read_sql_query(query, con=engine)

df_address

2025-01-12 10:26:16,926 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-01-12 10:26:16,928 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-12 10:26:16,931 INFO sqlalchemy.engine.Engine select current_schema()
2025-01-12 10:26:16,931 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-12 10:26:16,932 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-01-12 10:26:16,933 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-12 10:26:16,935 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-12 10:26:16,936 INFO sqlalchemy.engine.Engine Select * From Address;
2025-01-12 10:26:16,936 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-12 10:26:16,967 INFO sqlalchemy.engine.Engine ROLLBACK


,addressguid,addressid,region,state_province,city,postalcode,customerguid
0,458b5782-d58c-41f8-b43c-de2353afbb8a,1,Central,Illinois,Addison,60101,66379f73-c216-4450-bdd1-6f19512d5411
1,46bafd76-7ea4-49d2-ac66-4f25aed3afe9,2,West,Washington,Anacortes,98221,148d17dd-43f0-4855-9704-96257bf93292
2,31191c73-7d78-4d77-8430-29aade2702ef,3,West,California,San Gabriel,91776,12ac1694-41bc-48be-b1bc-a243c5f33c3d
3,f1828a20-24c2-421c-89c7-ace8112dbe28,4,West,California,San Jose,95123,c3778522-0212-4691-8204-40f6b716de79
4,10d96e09-3e46-4370-852b-8eafe0f89b5e,5,East,Massachusetts,Fall River,2724,19dadff2-cfd4-4e8c-b848-3066daea6f85
...,...,...,...,...,...,...,...
2698,bc2bbafc-f9ff-46ad-8a08-b5eae35c9408,2699,Central,Illinois,Decatur,62521,7d0f1c46-6ace-46b0-aabe-ed12cd87f006
2699,050afc22-f2ec-4656-839c-5b3aab27710c,2700,Central,Illinois,Des Plaines,60016,3aff70a4-1990-4f9a-abb1-67a9db1e85bb
2700,686c2f4f-0da2-46f3-940e-b6834437e5da,2701,East,West Virginia,Fairmont,26554,5dd3dbf4-a1a1-49c3-aa1a-21c94925adb3
2701,efff0b69-c379-4764-bf50-8cd6cbebd1b2,2702,East,West Virginia,Charleston,25314,3782402f-9d91-41fe-ab15-63f25972f082


In [ ]:
query = "Select * From \"Order\";"

df_order = pd.read_sql_query(query, con=engine)

df_order

In [18]:
query = "Select * From Shipping;"

df_shipping = pd.read_sql_query(query, con=engine)

df_shipping

2025-01-11 13:57:24,107 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-11 13:57:24,109 INFO sqlalchemy.engine.Engine Select * From Shipping;
2025-01-11 13:57:24,110 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-11 13:57:24,232 INFO sqlalchemy.engine.Engine ROLLBACK


,shippingguid,shippingid,dateshipped,shippingcost,shipmode,addressguid
0,08525375-068c-4d7d-b693-7fe7f4a74200,1,2012-05-30,$0.50,Regular Air,458b5782-d58c-41f8-b43c-de2353afbb8a
1,11587c1d-72e2-4b62-b8af-7661d4f22574,2,2010-07-08,$0.93,Express Air,46bafd76-7ea4-49d2-ac66-4f25aed3afe9
2,24fea7a4-c137-4481-a462-6fa3cc638318,3,2011-07-28,$6.15,Express Air,46bafd76-7ea4-49d2-ac66-4f25aed3afe9
3,1cea8854-90c3-45bd-8ad2-d96d7eeafb8f,4,2011-07-28,$3.60,Regular Air,46bafd76-7ea4-49d2-ac66-4f25aed3afe9
4,4db2ad1b-087f-43a9-80ba-e1e652c774fa,5,2011-07-27,$2.50,Express Air,46bafd76-7ea4-49d2-ac66-4f25aed3afe9
...,...,...,...,...,...,...
8885,e054c778-ecf7-40d0-84d4-ed104334b3a1,8886,2013-05-15,$14.72,Regular Air,efff0b69-c379-4764-bf50-8cd6cbebd1b2
8886,23e7c560-7791-4b95-bfac-52f80a811283,8887,2013-05-14,$7.49,Regular Air,efff0b69-c379-4764-bf50-8cd6cbebd1b2
8887,9773e161-5d5b-4103-a7ad-21ddbf3336e7,8888,2013-09-14,$45.70,Delivery Truck,efff0b69-c379-4764-bf50-8cd6cbebd1b2
8888,417ae906-de1e-4a8c-be3c-817121544187,8889,2010-02-11,$13.99,Express Air,2aac5d74-0a91-4ccd-abf7-1464e3a22f73


In [20]:
query = "Select * From Sales;"

df_sales = pd.read_sql_query(query, con=engine)

df_sales

2025-01-11 13:57:26,543 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-11 13:57:26,545 INFO sqlalchemy.engine.Engine Select * From Sales;
2025-01-11 13:57:26,546 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-11 13:57:26,614 INFO sqlalchemy.engine.Engine ROLLBACK


,salesguid,salesid,profit,sales,quantityordered,productbasemargin,discount,unitprice,orderguid,productguid
0,dbd698b1-5ede-4c17-9f94-40841ceac490,1,$1.32,$5.90,2,$0.36,0.01,$2.88,93d65299-326e-4a1e-b5ea-aa4282e099f2,e4ef7fb5-9a0a-4599-b82d-242ec77ebc52
1,9527e201-a6bf-46fd-af57-e2cfeace815a,2,$4.56,$13.01,4,$0.54,0.01,$2.84,9bc5d6ff-22c4-476d-a757-b0e994aebe17,ed8aa96c-b68f-42cc-b963-3680b23a92c4
2,1b69988a-eeb0-4871-adb9-f6bf96750996,3,-$47.64,$49.92,7,$0.37,0.03,$6.68,23cb8498-4c6a-476e-8e10-ee80a5dfb78f,738e6531-e2fb-4fdc-af0b-b58389f73e57
3,1fe1e186-ad8b-4e1f-a7e1-511c4f407524,4,"$1,388.05","$2,011.67",37,$0.37,0.09,$55.48,dbecd199-3705-48d4-aacd-68a093360aa6,738e6531-e2fb-4fdc-af0b-b58389f73e57
4,7aa780c2-7e2a-4d4c-ace1-df7ddc69668f,5,"$1,001.45","$1,451.37",12,$0.38,0.08,$120.97,59510e53-b44d-4029-82da-083af2854828,fef17e15-34a8-4859-88da-53fd3d3b55c5
...,...,...,...,...,...,...,...,...,...,...
7161,eed96b4f-6685-4ddf-ac5a-5a3e2b4d5210,7162,-$513.79,"$1,969.31",10,$0.64,0.08,$212.60,f1a81eee-7a55-4392-813a-1cc9144344a9,cbacf957-936d-4f95-9073-c07b4b3a0127
7162,b7d6bb4f-5877-4f64-b5e9-5fe226e20524,7163,$137.86,$447.87,13,$0.40,0.06,$35.89,e272280f-8abb-4b2a-abea-13879299941a,7e15d637-669b-443d-b198-dff51e955edb
7163,0b6c48a1-b3ba-40e3-8120-09e81ca0d917,7164,"-$1,225.03","$2,215.93",4,$0.71,0.08,$550.98,b8b32ec8-1b8b-4e5c-8493-37b0f112512a,cbacf957-936d-4f95-9073-c07b4b3a0127
7164,18539479-7048-4db9-8122-a0a8a8a59648,7165,$349.49,$506.50,5,$0.65,0.10,$105.98,e0ce9b4a-64a5-4914-b087-8fbbbc57cf29,939c8c26-98e7-47e8-8c41-2767bea8615a


In [22]:
limit = 5

In [24]:
class CustomerData:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def top_customers_by_segment(self, segment):
        return self.dataframe[self.dataframe['segment'] == segment].head(limit)

# Instantiate the class
customer_data = CustomerData(df_customer)

# Example method use
df_customer_data_top_segment = customer_data.top_customers_by_segment('Corporate')

In [26]:
df_customer_data_top_segment

,customerguid,customerid,fullname,segment
0,66379f73-c216-4450-bdd1-6f19512d5411,2,Janice Fletcher,Corporate
1,148d17dd-43f0-4855-9704-96257bf93292,3,Bonnie Potter,Corporate
18,04b3e1c4-4f12-4597-89d4-185e8fe43acb,23,Lynn Moss,Corporate
19,d5e25167-63cf-4885-9bc7-b7ca39c60d22,24,Edna Thomas,Corporate
20,6a411b01-853c-432d-8f03-d09092476243,25,Virginia Hardison,Corporate


In [28]:
class ProductData:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def top_products_by_category(self, category):
        return self.dataframe[self.dataframe['category'] == category].head(limit)

    def top_products_by_subcategory(self, subcategory):
        return self.dataframe[self.dataframe['subcategory'] == subcategory].head(limit)

    def top_products_by_container(self, container):
        return self.dataframe[self.dataframe['container'] == container].head(limit)

# Intantiate the class
product_data = ProductData(df_product)

# Example method use
df_product_data_top_category = product_data.top_products_by_category('Technology')

df_product_data_top_subcategory = product_data.top_products_by_subcategory('Paper')

df_product_data_top_container = product_data.top_products_by_container('Small Box')

In [30]:
df_product_data_top_category 

,productguid,category,subcategory,container,productid
4,1f08028e-660f-4bf9-af1b-c31274105e7c,Technology,Telephones and Communication,Small Box,5
5,fef17e15-34a8-4859-88da-53fd3d3b55c5,Technology,Office Machines,Jumbo Drum,6
8,253683ef-b61a-4996-9f07-34e8dece887a,Technology,Telephones and Communication,Wrap Bag,9
17,a67eb099-70af-48b6-b287-4a2ca61b4e54,Technology,Office Machines,Medium Box,18
21,3dad0962-6cd1-413c-82a9-bac0cf198420,Technology,Computer Peripherals,Small Pack,22


In [32]:
df_product_data_top_subcategory 

,productguid,category,subcategory,container,productid
2,738e6531-e2fb-4fdc-af0b-b58389f73e57,Office Supplies,Paper,Small Box,3
24,edf07270-3cad-4a3b-9c0a-5f7165e19bab,Office Supplies,Paper,Wrap Bag,25


In [34]:
df_product_data_top_container 

,productguid,category,subcategory,container,productid
0,e4ef7fb5-9a0a-4599-b82d-242ec77ebc52,Office Supplies,Labels,Small Box,1
2,738e6531-e2fb-4fdc-af0b-b58389f73e57,Office Supplies,Paper,Small Box,3
4,1f08028e-660f-4bf9-af1b-c31274105e7c,Technology,Telephones and Communication,Small Box,5
7,1e06b708-2c1a-4a1e-96eb-de2bddf080b9,Office Supplies,Appliances,Small Box,8
11,7e15d637-669b-443d-b198-dff51e955edb,Office Supplies,Envelopes,Small Box,12


In [36]:
class AddressData:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def top_address_by_region(self, region):
        return self.dataframe[self.dataframe['region'] == region].head(limit)

    def top_address_by_state_province(self, state_province):
        return self.dataframe[self.dataframe['state_province'] == state_province].head(limit)

    def top_address_by_city(self, city):
        return self.dataframe[self.dataframe['city'] == city].head(limit)

# Intantiate the class
address_data = AddressData(df_address)

df_address_data_top_region = address_data.top_address_by_region('Central')

df_address_data_top_state_province = address_data.top_address_by_state_province('California')

df_address_data_top_city = address_data.top_address_by_city('Fairmont')

In [38]:
df_address_data_top_region

,addressguid,addressid,region,state_province,city,postalcode,customerguid
0,458b5782-d58c-41f8-b43c-de2353afbb8a,1,Central,Illinois,Addison,60101,66379f73-c216-4450-bdd1-6f19512d5411
10,dc964880-86c7-4f5a-b1e2-0c8a280ee371,11,Central,Minnesota,Prior Lake,55372,bccd9fb4-78fd-4466-876e-5b0bc14e1c6a
16,12978c40-3b16-4fa3-8ce9-a51d50c7cb1f,17,Central,Nebraska,Bellevue,68005,9ae5989f-8cf0-407a-a3ab-1c8a80711282
31,ca733a7e-9716-4e26-aae4-5280a4e4a2a2,32,Central,Illinois,Alton,62002,2a458445-4d9c-4fce-a540-c4578318ba4c
35,9e8fe65e-6a2d-4acc-be5a-4dc4e4a3e44c,36,Central,Texas,Coppell,75019,bb5665ff-f225-4d34-b599-19391d862ddf


In [40]:
df_address_data_top_state_province

,addressguid,addressid,region,state_province,city,postalcode,customerguid
2,31191c73-7d78-4d77-8430-29aade2702ef,3,West,California,San Gabriel,91776,12ac1694-41bc-48be-b1bc-a243c5f33c3d
3,f1828a20-24c2-421c-89c7-ace8112dbe28,4,West,California,San Jose,95123,c3778522-0212-4691-8204-40f6b716de79
18,d0e63798-6ec5-485f-9e3f-905824d817c9,19,West,California,Laguna Hills,92653,04b3e1c4-4f12-4597-89d4-185e8fe43acb
19,626664b7-1619-4645-8816-f5af7cbb55fe,20,West,California,Laguna Niguel,92677,d5e25167-63cf-4885-9bc7-b7ca39c60d22
20,f6dd6bac-94ab-417b-bb49-da9fa08aeebf,21,West,California,Lake Elsinore,92530,6a411b01-853c-432d-8f03-d09092476243


In [42]:
df_address_data_top_city

,addressguid,addressid,region,state_province,city,postalcode,customerguid
2700,686c2f4f-0da2-46f3-940e-b6834437e5da,2701,East,West Virginia,Fairmont,26554,5dd3dbf4-a1a1-49c3-aa1a-21c94925adb3


In [44]:
class OrderData:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def top_order_by_priority(self, priority):
        return self.dataframe[self.dataframe['priority'] == priority].head(limit)

    def top_order_by_year_ordered(self, year):
        return self.dataframe[self.dataframe['dateordered'].dt.year == year].head(limit)

order_data = OrderData(df_order)

df_order_data_top_priority = order_data.top_order_by_priority('High')

df_order_data_top_year_ordered = order_data.top_order_by_year_ordered(2011)

In [46]:
df_order_data_top_priority

,orderguid,orderid,dateordered,priority,customerguid,shippingguid,productguid
1,9bc5d6ff-22c4-476d-a757-b0e994aebe17,2,2010-07-07,High,148d17dd-43f0-4855-9704-96257bf93292,11587c1d-72e2-4b62-b8af-7661d4f22574,ed8aa96c-b68f-42cc-b963-3680b23a92c4
7,11198cd8-948f-4c4b-bcff-81fdd01a1584,8,2011-05-26,High,c3778522-0212-4691-8204-40f6b716de79,c59396dd-ad56-4079-8432-fee3bb3212b6,1e06b708-2c1a-4a1e-96eb-de2bddf080b9
9,9bf27ad0-d439-47b8-8488-27b10e0d084b,10,2012-10-30,High,19dadff2-cfd4-4e8c-b848-3066daea6f85,31c19b30-9460-40d1-9cf2-c786c95dd5e5,738e6531-e2fb-4fdc-af0b-b58389f73e57
14,cfe8b2a5-9f90-4c06-b8b1-bca67d2de1ef,15,2012-02-24,High,48d945a9-29df-4702-90dc-02ec9c1761c2,8be1e49e-3e13-48ba-a0f3-b451612551f0,a5394cfa-30a4-4615-962a-df36baf93f85
23,7d08899a-2c46-4c41-bbc6-9b2bbfb3ef34,24,2010-02-12,High,d8782f22-0cd2-41a3-8516-9227f3eafc0d,9a3f14aa-4a84-4cc7-8602-256464eb12f8,c4b73899-c971-41f4-9f01-faaccaf64a66


In [48]:
df_order_data_top_year_ordered

,orderguid,orderid,dateordered,priority,customerguid,shippingguid,productguid
2,23cb8498-4c6a-476e-8e10-ee80a5dfb78f,3,2011-07-27,Not Specified,148d17dd-43f0-4855-9704-96257bf93292,24fea7a4-c137-4481-a462-6fa3cc638318,738e6531-e2fb-4fdc-af0b-b58389f73e57
3,dbecd199-3705-48d4-aacd-68a093360aa6,4,2011-11-09,Medium,148d17dd-43f0-4855-9704-96257bf93292,1a8d4763-54e8-427b-9539-3024b8096dc5,738e6531-e2fb-4fdc-af0b-b58389f73e57
7,11198cd8-948f-4c4b-bcff-81fdd01a1584,8,2011-05-26,High,c3778522-0212-4691-8204-40f6b716de79,c59396dd-ad56-4079-8432-fee3bb3212b6,1e06b708-2c1a-4a1e-96eb-de2bddf080b9
12,ee5f9fda-9cab-4f9f-a7ec-93f2da649844,13,2011-10-04,Critical,ce5df26f-4c2e-4edb-8727-d138d7970b53,f72f7ef3-6ca2-49d7-8017-3638e22eb806,7e15d637-669b-443d-b198-dff51e955edb
19,69291e01-ba0a-4c84-bc8a-1aebe668731e,20,2011-12-04,Medium,cfa99880-fd56-44a6-8863-64ffd598fb67,9168609d-ce37-423b-86a7-18a671022eef,ed8aa96c-b68f-42cc-b963-3680b23a92c4


In [54]:
class ShippingData:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def top_shipping_by_shipmode(self, shipmode):
        return self.dataframe[self.dataframe['shipmode'] == shipmode].head(limit)

    def top_shipping_by_year_shipped(self, year):
        return self.dataframe[self.dataframe['dateshipped'].dt.year == year].head(limit)

shipping_data = ShippingData(df_shipping)

df_shipping_data_top_shipmode = shipping_data.top_shipping_by_shipmode('Regular Air')

df_shipping_data_top_year_shipped = shipping_data.top_shipping_by_year_shipped(2012)

In [52]:
df_shipping_data_top_shipmode

,shippingguid,shippingid,dateshipped,shippingcost,shipmode,addressguid
0,08525375-068c-4d7d-b693-7fe7f4a74200,1,2012-05-30,$0.50,Regular Air,458b5782-d58c-41f8-b43c-de2353afbb8a
3,1cea8854-90c3-45bd-8ad2-d96d7eeafb8f,4,2011-07-28,$3.60,Regular Air,46bafd76-7ea4-49d2-ac66-4f25aed3afe9
8,33ea719c-a56c-4362-987b-982a3f2dcc3e,9,2012-05-21,$6.81,Regular Air,31191c73-7d78-4d77-8430-29aade2702ef
9,c59396dd-ad56-4079-8432-fee3bb3212b6,10,2011-05-26,$0.99,Regular Air,f1828a20-24c2-421c-89c7-ace8112dbe28
10,73b0459c-4c1d-448a-8b1d-ffffdc62135a,11,2012-12-31,$6.60,Regular Air,f1828a20-24c2-421c-89c7-ace8112dbe28


In [56]:
df_shipping_data_top_year_shipped

,shippingguid,shippingid,dateshipped,shippingcost,shipmode,addressguid
0,08525375-068c-4d7d-b693-7fe7f4a74200,1,2012-05-30,$0.50,Regular Air,458b5782-d58c-41f8-b43c-de2353afbb8a
8,33ea719c-a56c-4362-987b-982a3f2dcc3e,9,2012-05-21,$6.81,Regular Air,31191c73-7d78-4d77-8430-29aade2702ef
10,73b0459c-4c1d-448a-8b1d-ffffdc62135a,11,2012-12-31,$6.60,Regular Air,f1828a20-24c2-421c-89c7-ace8112dbe28
11,31c19b30-9460-40d1-9cf2-c786c95dd5e5,12,2012-10-31,$4.86,Regular Air,10d96e09-3e46-4370-852b-8eafe0f89b5e
12,2667d448-29a0-49a2-a5fb-850659bf7eb8,13,2012-12-26,$36.09,Delivery Truck,d2e79bd6-0107-47ef-aecd-7dd20bb777bd


In [76]:
class SalesData:

    def __init__(self, dataframe):
        self.dataframe = dataframe

    def top_sales_by_profit(self, profit):
        return self.dataframe[self.dataframe['profit'] >= profit].head(limit)

    def top_sales_by_salesamount(self, sales):
        return self.dataframe[self.dataframe['sales'] >= sales].head(limit)

    def top_sales_by_quantity(self, quantity):
        return self.dataframe[self.dataframe['quantityordered'] >= quantity].head(limit)

sales_data = SalesData(df_sales)

df_sales_data_top_profit = sales_data.top_sales_by_profit('$2')

df_sales_data_top_salesamount = sales_data.top_sales_by_salesamount('$100')

df_sales_data_top_quantity = sales_data.top_sales_by_quantity(7)

In [72]:
df_sales_data_top_profit

,salesguid,salesid,profit,sales,quantityordered,productbasemargin,discount,unitprice,orderguid,productguid
1,9527e201-a6bf-46fd-af57-e2cfeace815a,2,$4.56,$13.01,4,$0.54,0.01,$2.84,9bc5d6ff-22c4-476d-a757-b0e994aebe17,ed8aa96c-b68f-42cc-b963-3680b23a92c4
2,1b69988a-eeb0-4871-adb9-f6bf96750996,3,-$47.64,$49.92,7,$0.37,0.03,$6.68,23cb8498-4c6a-476e-8e10-ee80a5dfb78f,738e6531-e2fb-4fdc-af0b-b58389f73e57
5,39eb1f32-8c40-47b7-8848-b2f08b7ceca4,6,"$4,390.37","$6,362.85",12,$0.60,0.02,$500.98,9e7eaefe-b80f-4019-89a9-a551bcaf7238,38d5a9ce-872b-42f2-ace5-a04dff5365bc
6,1e51a607-5eb3-427c-b234-c91f7c668cfd,7,-$141.26,$113.25,18,$0.36,0.08,$6.48,9868ede2-1d7b-44e5-a38c-d2546aa76d50,738e6531-e2fb-4fdc-af0b-b58389f73e57
8,9968e7d9-836f-45e1-b39f-3732dacdb1ef,9,-$13.86,$28.61,4,$0.37,0.07,$6.48,9e56fea8-c617-47ca-8bcb-5821bb07101b,738e6531-e2fb-4fdc-af0b-b58389f73e57


In [78]:
df_sales_data_top_salesamount

,salesguid,salesid,profit,sales,quantityordered,productbasemargin,discount,unitprice,orderguid,productguid
0,dbd698b1-5ede-4c17-9f94-40841ceac490,1,$1.32,$5.90,2,$0.36,0.01,$2.88,93d65299-326e-4a1e-b5ea-aa4282e099f2,e4ef7fb5-9a0a-4599-b82d-242ec77ebc52
1,9527e201-a6bf-46fd-af57-e2cfeace815a,2,$4.56,$13.01,4,$0.54,0.01,$2.84,9bc5d6ff-22c4-476d-a757-b0e994aebe17,ed8aa96c-b68f-42cc-b963-3680b23a92c4
2,1b69988a-eeb0-4871-adb9-f6bf96750996,3,-$47.64,$49.92,7,$0.37,0.03,$6.68,23cb8498-4c6a-476e-8e10-ee80a5dfb78f,738e6531-e2fb-4fdc-af0b-b58389f73e57
3,1fe1e186-ad8b-4e1f-a7e1-511c4f407524,4,"$1,388.05","$2,011.67",37,$0.37,0.09,$55.48,dbecd199-3705-48d4-aacd-68a093360aa6,738e6531-e2fb-4fdc-af0b-b58389f73e57
5,39eb1f32-8c40-47b7-8848-b2f08b7ceca4,6,"$4,390.37","$6,362.85",12,$0.60,0.02,$500.98,9e7eaefe-b80f-4019-89a9-a551bcaf7238,38d5a9ce-872b-42f2-ace5-a04dff5365bc


In [80]:
df_sales_data_top_quantity

,salesguid,salesid,profit,sales,quantityordered,productbasemargin,discount,unitprice,orderguid,productguid
2,1b69988a-eeb0-4871-adb9-f6bf96750996,3,-$47.64,$49.92,7,$0.37,0.03,$6.68,23cb8498-4c6a-476e-8e10-ee80a5dfb78f,738e6531-e2fb-4fdc-af0b-b58389f73e57
3,1fe1e186-ad8b-4e1f-a7e1-511c4f407524,4,"$1,388.05","$2,011.67",37,$0.37,0.09,$55.48,dbecd199-3705-48d4-aacd-68a093360aa6,738e6531-e2fb-4fdc-af0b-b58389f73e57
4,7aa780c2-7e2a-4d4c-ace1-df7ddc69668f,5,"$1,001.45","$1,451.37",12,$0.38,0.08,$120.97,59510e53-b44d-4029-82da-083af2854828,fef17e15-34a8-4859-88da-53fd3d3b55c5
5,39eb1f32-8c40-47b7-8848-b2f08b7ceca4,6,"$4,390.37","$6,362.85",12,$0.60,0.02,$500.98,9e7eaefe-b80f-4019-89a9-a551bcaf7238,38d5a9ce-872b-42f2-ace5-a04dff5365bc
6,1e51a607-5eb3-427c-b234-c91f7c668cfd,7,-$141.26,$113.25,18,$0.36,0.08,$6.48,9868ede2-1d7b-44e5-a38c-d2546aa76d50,738e6531-e2fb-4fdc-af0b-b58389f73e57


In [82]:
# Optionally, dispose the engine if it's no longer needed
engine.dispose()